## C. Les Machines Abstraites

Une **machine abstraite** est un modèle théorique d'un ordinateur. Il importe peu, dans ce cadre, de savoir si cet appareil peut effectivement être construit, mais plutôt d'appréhender, par ce modèle simplifié, le fonctionnement des machines, et de les comparer entre elles.

En général, une machine abstraite consiste en données d'entrée, en données de sortie et en un ensemble d'opérations autorisées pour transformer les entrées en sorties. On peut considérer les possibilités de transitions internes qui régissent le comportement d'une machine comme le programme informatique qui la dirige.

La **[machine de Turing](https://fr.wikipedia.org/wiki/Machine_de_Turing)** est historiquement la première machine abstraite imaginée et c'est par conséquent la plus connue.

Des définitions plus complexes consistent en des machines abstraites avec un jeu d'instructions complet, des registres et un modèle de mémoire. Un tel modèle fréquemment mentionné est la **Random Access Machine** (ou [RAM](https://en.wikipedia.org/wiki/Random-access_machine)) qui permet un accès direct à la mémoire. Dans les machine réelles modernes, l'accès à la mémoire externe, ou à la mémoire en cache, joue un rôle croissant dans la conception d'algorithmes.

### I. La Machine de Turing (TM)

Plusieurs définitions formelles proches les unes des autres peuvent être données d'une machine de Turing. L'une d'elles, relativement courante, est choisie ici. Une machine de Turing est un quintuplet $\displaystyle (Q,\Gamma ,q_{0},\delta ,F)$ où:

 * $Q$ est un ensemble fini d'états,
 * $\Gamma$ est l'alphabet de travail des symboles de la bande, contenant $B$ un symbole particulier (dit blanc), $B\in \Gamma$,
 * $q_0$ est l'état initial, $q_{0} \in Q$,
 * $\delta :Q\times \Gamma \to Q\times \Gamma \times \{\leftarrow ,\rightarrow \}$ est la fonction de transition,
 * $F$ est l'ensemble des états acceptants (ou finals, terminaux), $F\subseteq Q$.

Vous pouvez voir une machine de Turing mécanique en action! Youtube: https://www.youtube.com/watch?v=ivPv_kaYuwk

#### Implémentation

In [2]:
"""This module provides an implementation of a Turing Machine.
"""
import time
from enum import Enum
from collections import defaultdict
from typing import Generic, Type, TypeVar, Container, Iterable, Mapping, Tuple


State = TypeVar("Q")
Alphabet = TypeVar("Γ", bound=Enum)


class D(int, Enum):
    """The displacement allowed on the tape."""

    L = -1
    R = +1


class TuringMachine(Generic[State, Alphabet]):
    """A Turing Machine.

    Attributes:
        Q: The possible states (given as an `Enum`).
        Γ: The alphabet (given as an `Enum`).
        q0: The initial state.
        δ: The transitions.
        F: The final states.
        _tape: The tape of the Turing Machine.
        _current_state: The current state of the Turing Machine.
        _tape_index: The current position in the tape of the Turing Machine.
        _max_index: The rightmost position in the tape ever reached.
        _min_index: The leftmost position in the tape ever reached.

    Note:
        We represent the tape as a defaultdict where the default value is the
        blank character. This allow us to simulate an infite tape easily.
        However we need a method (here we use a `@property`) to convert the
        tape from a string, that is manipulated easily by the end user to the
        default dict.
    """

    def __init__(
        self,
        *,
        Q: Type[State],
        Γ: Type[Alphabet],
        q0: State,
        δ: Mapping[Tuple[State, Alphabet], Tuple[State, Alphabet, D]],
        F: Container[State],
    ) -> None:
        """Create an instance of the Turing Machine.

        Args:
            Q: The possible states (given as an `Enum`).
            Γ: The alphabet (given as an `Enum`).
            q0: The initial state.
            δ: The transitions.
            F: The final states.
        """
        self.Q = Q
        self.Γ = Γ
        self.δ = δ
        self.F = F
        self.q0 = q0

        self.reset()

    def reset(self) -> None:
        """Reset the Turing Machine internal state."""
        self._tape = defaultdict(lambda: self.Γ.B)
        self._current_state = self.q0
        self._max_index = 0
        self._min_index = 0
        self._tape_index = 0

    def step(self) -> None:
        """Run a step of the Turing Machine."""
        (
            current_state_unchecked,
            self._tape[self._tape_index],
            displacement_unchecked,
        ) = self.δ[(self._current_state, self._tape[self._tape_index])]
        self._current_state = self.Q(current_state_unchecked)
        self._displacement = D(displacement_unchecked)
        self._tape_index += self._displacement

    @property
    def tape(self) -> str:
        """Represent the tape as a string."""
        return "".join(
            self.Γ(self._tape[i]).value
            for i in range(self._min_index, self._max_index + 1)
        ).strip(Γ.B.value)

    @tape.setter
    def tape(self, tape: str) -> None:
        """Set the tape from a string."""
        for i, symbol in enumerate(tape):
            self._tape[i] = Γ(symbol)

    @property
    def cnt(self) -> bool:
        """Check if the curent state is in a final state.

        Returns:
            True if the current state is not in a final state, False otherwise.
        """
        return self._current_state not in self.F

    def run(self, tape: Iterable[Alphabet], *, freq: float) -> int:
        """Run the Turing Machine until it reaches on of the final state(s).

        Args:
            freq: the frequency (speed) of computation of the Turing Machine.

        Returns:
            The number of cycles perform until the machine halts.
        """
        self.tape = tape

        max_nap_time = 1 / freq
        cycles = 0
        while self.cnt:
            work_time = -time.monotonic()
            self.step()
            self._max_index = max(self._tape_index, self._max_index)
            self._min_index = min(self._tape_index, self._min_index)
            cycles += 1
            work_time += time.monotonic()
            time.sleep(max(max_nap_time - work_time, 0))

        return cycles

In [15]:
# Turing Machine, identity
class Q(str, Enum):
    "The states of the Turing Machine."
    INITIAL = "I"
    FINAL = "F"


class Γ(str, Enum):
    """The alphabet of the Turing Machine."""

    ZERO = "0"
    ONE = "1"
    B = " "


turing_machine = TuringMachine(
    Q=Q,
    Γ=Γ,
    q0=Q.INITIAL,
    F={Q.FINAL},
    δ={
        (Q.INITIAL, Γ.ZERO): (Q.INITIAL, Γ.ZERO, D.R),
        (Q.INITIAL, Γ.ONE): (Q.INITIAL, Γ.ONE, D.R),
        (Q.INITIAL, Γ.B): (Q.FINAL, Γ.B, D.R),
    },
)

In [16]:
cycles = turing_machine.run("010", freq=100.0)
print(turing_machine.tape, "en", cycles, "cycles.")

010 en 4 cycles.


#### 📖 Exercice

 * Écrire un programme qui change les 0 en 1 d'un ruban binaire

In [13]:
# Turing Machine, replacing zeros by ones
class Q(str, Enum):
    "The states of the Turing Machine."

    INITIAL = "I"
    FINAL = "F"


class Γ(str, Enum):
    """The alphabet of the Turing Machine."""

    ZERO = "0"
    ONE = "1"
    B = " "


turing_machine = TuringMachine(
    Q=Q,
    Γ=Γ,
    q0="I",
    F={"F"},
    δ={
        ("I", "0"): ("I", "1", +1),
        ("I", "1"): ("I", "0", +1),
        ("I", " "): ("F", " ", +1),
    },
)

In [14]:
cycles = turing_machine.run("010", freq=1.0)
print(turing_machine.tape, "en", cycles, "cycles.")

101 en 4 cycles.


#### 📖 Exercice

 * Écrire un programme qui affiche "Hello, World!" à partir d'un ruban vide.

In [11]:
# Turing Machine, print "Hello, World!"
class Q(str, Enum):
    """The states of the Turing Machine.

    We use 14 states (one per letter in "Hello, World!") plus a final state.
    """

    C01 = "C01"
    C02 = "C02"
    C03 = "C03"
    C04 = "C04"
    C05 = "C05"
    C06 = "C06"
    C07 = "C07"
    C08 = "C08"
    C09 = "C09"
    C10 = "C10"
    C11 = "C11"
    C12 = "C12"
    C13 = "C13"
    C14 = "C14"


class Γ(str, Enum):
    """The alphabet of the Turing Machine."""

    H = "H"
    E = "E"
    L = "L"
    O = "O"
    COMMA = ","
    W = "W"
    R = "R"
    D = "D"
    EXCLAMATION_MARK = "!"
    B = " "


turing_machine = TuringMachine(
    Q=Q,
    Γ=Γ,
    q0="C01",
    F={"C14"},
    δ={
        ("C01", " "): ("C02", "H", +1),
        ("C02", " "): ("C03", "E", +1),
        ("C03", " "): ("C04", "L", +1),
        ("C04", " "): ("C05", "L", +1),
        ("C05", " "): ("C06", "O", +1),
        ("C06", " "): ("C07", ",", +1),
        ("C07", " "): ("C08", " ", +1),
        ("C08", " "): ("C09", "W", +1),
        ("C09", " "): ("C10", "O", +1),
        ("C10", " "): ("C11", "R", +1),
        ("C11", " "): ("C12", "L", +1),
        ("C12", " "): ("C13", "D", +1),
        ("C13", " "): ("C14", "!", +1),
    },
)

In [12]:
cycles = turing_machine.run("", freq=100.0)
print(turing_machine.tape, "en", cycles, "cycles.")

HELLO, WORLD! en 13 cycles.


#### 📖 Exercice

 * [Difficile] Écrire un programme qui compte le nombre de zéros d'un ruban
   
   par example 000000 -> 011 (7 en binaire, little-endian)

In [4]:
# Turing Machine, count zeros
class Q(str, Enum):
    """The states of the Turing Machine."""

    SEARCH_END_PAIR = "SEARCH_END_PAIR"
    SEARCH_END_ODD = "SEARCH_END_ODD"
    PRINT_PAIR = "PRINT_PAIR"
    PRINT_ODD = "PRINT_ODD"
    BACKTRACK = "BACKTRACK"
    CHECK_END = "CHECK_END"
    RESTART = "RESTART"
    CLEAN = "CLEAN"
    FINAL = "FINAL"


class Γ(str, Enum):

    """The alphabet of the Turing Machine."""

    Z = "0"
    O = "1"
    D = "-"
    B = " "

turing_machine = TuringMachine(
    Q=Q,
    Γ=Γ,
    q0="SEARCH_END_PAIR",
    F={"FINAL"},
    δ={
        ("SEARCH_END_PAIR", "0"): ("SEARCH_END_ODD", "-", +1),
        ("SEARCH_END_PAIR", "-"): ("SEARCH_END_PAIR", "-", +1),
        ("SEARCH_END_PAIR", " "): ("PRINT_PAIR", " ", +1),
        ("SEARCH_END_ODD", "0"): ("SEARCH_END_PAIR", "0", +1),
        ("SEARCH_END_ODD", "-"): ("SEARCH_END_ODD", "-", +1),
        ("SEARCH_END_ODD", " "): ("PRINT_ODD", " ", +1),
        ("PRINT_PAIR", "0"): ("PRINT_PAIR", "0", +1),
        ("PRINT_PAIR", "1"): ("PRINT_PAIR", "1", +1),
        ("PRINT_PAIR", " "): ("BACKTRACK", "0", -1),
        ("PRINT_ODD", "0"): ("PRINT_ODD", "0", +1),
        ("PRINT_ODD", "1"): ("PRINT_ODD", "1", +1),
        ("PRINT_ODD", " "): ("BACKTRACK", "1", -1),
        ("BACKTRACK", "0"): ("BACKTRACK", "0", -1),
        ("BACKTRACK", "1"): ("BACKTRACK", "1", -1),
        ("BACKTRACK", " "): ("CHECK_END", " ", -1),
        ("CHECK_END", "-"): ("CHECK_END", "-", -1),
        ("CHECK_END", "0"): ("RESTART", "0", -1),
        ("CHECK_END", " "): ("CLEAN", " ", +1),
        ("RESTART", "0"): ("RESTART", "0", -1),
        ("RESTART", "-"): ("RESTART", "-", -1),
        ("RESTART", " "): ("SEARCH_END_PAIR", " ", +1),
        ("CLEAN", "-"): ("CLEAN", " ", +1),
        ("CLEAN", " "): ("FINAL", " ", +1),
    },
)

In [5]:
cycles = turing_machine.run("0000000", freq=1000.0)
print(
    # Python conversion to in suppose the binary string is Big Endian (the
    # 'natural' notation for human). However our turing machine return the
    # result as Little Endian, so we must reverse it (using [::1]) before
    # conversion.
    int(turing_machine.tape[::-1], 2),
    "en",
    cycles,
    "cycles.",
)

7 en 68 cycles.
